# Bokeh

## John Wilson

In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_file, show, Figure, output_notebook
from bokeh.models import ColumnDataSource, WMTSTileSource, HoverTool, Select
from bokeh.io import output_file, show, curdoc
from bokeh.models.widgets import Select
from bokeh.layouts import column
from bokeh.palettes import Reds9
import pickle
from pyproj import Proj, transform

### Problem 1

In [2]:
accidents = drivers=pd.read_pickle('fars_data/final_accidents2.pickle')

In [3]:
drivers = pd.read_pickle('fars_data/final_drivers.pickle')

In [4]:
#drivers=pd.read_pickle('fars_data/final_drivers.pickle')
accidents[accidents["LATITUDE"]==77.7777]=np.nan
accidents[accidents["LATITUDE"]==88.8888]=np.nan
accidents[accidents["LATITUDE"]==99.9999]=np.nan
accidents[accidents["LONGITUD"]==777.7777]=np.nan
accidents[accidents["LONGITUD"]==888.8888]=np.nan
accidents[accidents["LONGITUD"]==999.9999]=np.nan
accidents=accidents.dropna()

### Problem 2

In [5]:
from_proj = Proj(init="epsg:4326")
to_proj = Proj(init="epsg:3857")

def convert(longitudes, latitudes):
    """Converts latlon coordinates to meters.
    
    Inputs:
        longitudes (array-like) : array of longitudes
        latitudes (array-like) : array of latitudes
        
    Example:
        x,y = convert(accidents.LONGITUD, accidents.LATITUDE)
    """
    x_vals = []
    y_vals = []
    for lon, lat in zip(longitudes, latitudes):
        x, y = transform(from_proj, to_proj, lon, lat)
        x_vals.append(x)
        y_vals.append(y)
    return x_vals, y_vals

accidents["x"], accidents["y"] = convert(accidents.LONGITUD, accidents.LATITUDE)

### Problem 4

In [6]:
fig = Figure(plot_width=1100, plot_height=650,
    x_range=(-13000000, -7000000), y_range=(2750000, 6250000),
    tools=["wheel_zoom", "pan"], active_scroll="wheel_zoom", webgl=True)
fig.axis.visible = False

STAMEN_TONER_BACKGROUND = WMTSTileSource(
    url='http://tile.stamen.com/toner-background/{Z}/{X}/{Y}.png',
    attribution=(
        'Map tiles by <a href="http://stamen.com">Stamen Design</a>, '
        'under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>.'
        'Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, '
        'under <a href="http://www.openstreetmap.org/copyright">ODbL</a>'
    )
)

fig.add_tile(STAMEN_TONER_BACKGROUND)

TileRenderer(id='071843de-55e1-4204-97f2-9d691fc0007f', ...)

### Problem 6

In [7]:
cir_source = ColumnDataSource(accidents[(accidents["SP"]==0)&(accidents["DRUNK_DR"]==0)])
cir = fig.circle(x='x', y='y', source=cir_source, fill_color='green', \
                 line_width=0, size=1, fill_alpha=.5)

cir_source = ColumnDataSource(accidents[(accidents["SP"]==1)&(accidents["DRUNK_DR"]==0)])
cir1 = fig.circle(x='x', y='y', source=cir_source, fill_color='red', \
                 line_width=0, size=1, fill_alpha=.5)

cir_source = ColumnDataSource(accidents[(accidents["SP"]==0)&(accidents["DRUNK_DR"]==1)])
cir2 = fig.circle(x='x', y='y', source=cir_source, fill_color='blue', \
                 line_width=0, size=1, fill_alpha=.5)

cir_source = ColumnDataSource(accidents[(accidents["SP"]==1)&(accidents["DRUNK_DR"]==1)])
cir3 = fig.circle(x='x', y='y', source=cir_source, fill_color='purple', \
                 line_width=0, size=1, fill_alpha=.5)

In [8]:
show(fig)